In [1]:
TaskObject = ""

In [82]:
import requests
from requests.auth import HTTPDigestAuth
from notebookutils import mssparkutils
import re
import json
import pandas as pd
from datetime import date
from datetime import datetime

#TaskObject = "{\"TaskInstanceId\":8,\"TaskMasterId\":6,\"TaskStatus\":\"InProgress\",\"TaskType\":\"REST API to Azure Storage\",\"Enabled\":1,\"ExecutionUid\":\"7a050f7a-5a32-4e49-8fdd-e70fcc5269d8\",\"NumberOfRetries\":1,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https:// ark-stg-kv-ads-ikjn.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-2,\"EngineName\":\"arkstgsynwadsikjn\",\"SystemType\":\"Synapse\",\"ResourceGroup\":\"LockboxDev01\",\"SubscriptionId\":\"687fe1ae-a520-4f86-b921-a80664c40f9b\",\"ADFPipeline\":\"GPL_SparkNotebookExecution_Azure\",\"EngineJson\":\"{\\r\\n            \\\"endpoint\\\": \\\"https://arkstgsynwadsikjn.dev.azuresynapse.net\\\", \\\"DeltaProcessingNotebook\\\": \\\"DeltaProcessingNotebook\\\", \\\"PurviewAccountName\\\": \\\"dlzdev07purv\\\", \\\"DefaultSparkPoolName\\\":\\\"Dummy\\\"\\r\\n        }\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{\"endpoint\":\"https://arkstgsynwadsikjn.dev.azuresynapse.net\",\"DeltaProcessingNotebook\":\"DeltaProcessingNotebook\",\"PurviewAccountName\":\"dlzdev07purv\",\"DefaultSparkPoolName\":\"Dummy\"}},\"Source\":{\"System\":{\"SystemId\":-19,\"SystemServer\":\"(baseurl?)\",\"AuthenticationType\":\"MSI\",\"Type\":\"Rest\",\"Username\":null,\"BaseUrl\":\"https://api.hubapi.com\",\"NextPageUrlKey\":\"link\",\"headers\":\"{}\"},\"Instance\":{\"TargetRelativePath\":\"{y}/{m}/{d}/\"},\"Pagination\":\"NextPageUrlKey\",\"RelativeUrl\":\"/crm/v3/objects/contacts?limit=100&properties=firstname%2Clastname%2Cemail%2Cphone%2Ccompany&hapikey=$InsertSecret($KeyVault(ark-stg-kv-ads-ikjn).$SecretName(HubSpotAPIToken).$LinkedServiceName(SLS_AzureKeyVault))$END\",\"RequestBody\":\"\",\"RequestMethod\":\"GET\",\"Type\":\"Rest\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsikjnadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"{y}/{m}/{d}/\"},\"DataFileName\":\"TestHubSpot\",\"PartitionSize\":\"Multiple\",\"RelativePath\":\"\",\"Type\":\"Json\"},\"TMOptionals\":{\"ExecuteNotebook\":\"RestAPINotebook\",\"Purview\":\"Disabled\",\"QualifiedIDAssociation\":\"TaskMasterId\",\"UseNotebookActivity\":\"Enabled\"}}"
TaskObjectJson = json.loads(TaskObject)
#Creating Headers
headers = TaskObjectJson["Source"]["System"]["headers"]
#print(headers)
headers = json.loads(headers)
for a,b in headers.items():
    if ("$InsertSecret" in b):
        result = re.search('\$KeyVault\((.*)\)\.\$Secret', b)
        keyVaultName = (result.group(1))
        result = re.search('\$SecretName\((.*)\)\.', b)
        secretName = (result.group(1))
        result = re.search('\$LinkedServiceName\((.*)\)\)', b)
        linkedServiceName = (result.group(1))
        secret = TokenLibrary.getSecret(keyVaultName, secretName, linkedServiceName)
        headers[a] = re.match("^.*(?=(\$InsertSecret))",b).group() + secret
#print(headers)


#Create Request Body
if(TaskObjectJson["Source"]["RequestBody"] == "" or TaskObjectJson["Source"]["RequestMethod"] == "GET"):
    payload = {}
else:
    payload = json.loads(TaskObjectJson["Source"]["RequestBody"])
    payload = json.dumps(payload)


#Get RequestMethod / Set up Full URL
#print(payload)
requestMethod = TaskObjectJson["Source"]["RequestMethod"]


url = TaskObjectJson["Source"]["System"]["BaseUrl"] + TaskObjectJson["Source"]["RelativeUrl"]
#url = baseurl + relativeurl
#Search URL for any secret
if ("$InsertSecret" in url):
    result = re.search('\$KeyVault\((.*)\)\.\$Secret', url)
    keyVaultName = (result.group(1))
    result = re.search('\$SecretName\((.*)\)\.', url)
    secretName = (result.group(1))
    result = re.search('\$LinkedServiceName\((.*)\)\)', url)
    linkedServiceName = (result.group(1))
    secret = TokenLibrary.getSecret(keyVaultName, secretName, linkedServiceName)
    urlpt1 = re.match("^.*(?=(\$InsertSecret))",url).group() + secret
    urlpt2 = url.split("$END")[1]
    url = urlpt1 + urlpt2
 
#print(url)
response = requests.request(requestMethod, url, headers=headers, data=payload)
#print(response.content)

now = datetime.now()

#TARGET
Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"
Target = Target + TaskObjectJson['Target']['Instance']['TargetRelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']
Target = Target.replace('//', '/')
Target = "abfss://" + Target
Target = Target.replace("{yyyy}", "%Y")
Target = Target.replace("{MM}", "%m")
Target = Target.replace("{dd}", "%d")
Target = Target.replace("{hh}", "%H")
Target = Target.replace("{mm}", "%M")
Target = now.strftime(Target)
print("SAVE TARGET: " + Target)
partitionSize = TaskObjectJson['Target']['PartitionSize']
jData = json.loads(response.content)

pagination = TaskObjectJson["Source"]["Pagination"]
#PAGINATION - This is modified from the default one.
if(pagination == "NextPageUrlKey"):
    #we find our next page key
    nextPageKey = TaskObjectJson["Source"]["System"]["NextPageUrlKey"]
    nextPageUrl = ""
    print(nextPageKey)
    json_data_list = []
    paginationCheck = True

    #jData = json.loads(response.content)
    json_data_list.append(jData)
    while(paginationCheck):
        if("paging" in jData):
            paginationUrl = jData['paging']['next'][nextPageKey]
            #print(paginationUrl)
            paginationUrl = paginationUrl + "&hapikey=" + secret
            #print(paginationUrl)
            response = requests.request(requestMethod, paginationUrl, headers=headers, data=payload)
            jData = json.loads(response.content)
            json_data_list.append(jData)
        else:
            paginationCheck = False
    #Create our neat json file
    fullList = []
    for ls in json_data_list:
        for res in ls['results']:
            for key, value in res['properties'].items():
                res[key] = value
            res.pop('properties')
            #res.pop('hs_object_id')
            fullList.append(res)

    
    fullJson = { "combinedJson": fullList }
    fullJson = json.dumps(fullJson)
    df = spark.read.json(sc.parallelize([fullJson]))
    if(partitionSize == "Multiple"):
        df.write.format("json").mode("overwrite").save(Target)
    else:
        df.write.format("json").mode("overwrite").coalesce(1).save(Target)

else:
    #SAVE RESPONSE
    df = spark.read.json(sc.parallelize([jData]))
    if(partitionSize == "Multiple"):
        df.write.format("json").mode("overwrite").save(Target)
    else:
        df.write.format("json").mode("overwrite").coalesce(1).save(Target)
